In [1]:
import xarray as xr
from distributed import LocalCluster
import json
from pathlib import Path
import folium
from datetime import datetime
import s3fs
from collections import defaultdict

In [2]:
bucket = "e05ab01a9d56408d82ac32d69a5aae2a:sample-data"
prefixes = ["tutorial_data/cpm_v253/", "tutorial_data/cpm_v256/"]
prefix_url = "https://objectstore.eodc.eu:2222"
start_date = datetime(2023, 6, 1)
end_date = datetime(2023, 10, 30)

fs = s3fs.S3FileSystem(anon=True, client_kwargs={"endpoint_url": prefix_url})

# Unregister CEPH handler (required for EODC)
handlers = fs.s3.meta.events._emitter._handlers
handlers_to_unregister = handlers.prefix_search("before-parameter-build.s3")
fs.s3.meta.events._emitter.unregister("before-parameter-build.s3", handlers_to_unregister[0])

filtered_urls = []

for prefix in prefixes:
    # Find all potential Sentinel-3 LST datasets
    s3_pattern = f"s3://{bucket}/{prefix}S3[AB]_SL_2_LST____*.zarr"
    candidates = fs.glob(s3_pattern)
    
    for s3_path in candidates:
        # Convert S3 path to HTTP URL
        http_url = f"{prefix_url}/{s3_path.replace('s3://', '')}"
        
        # Extract filename from URL
        filename = s3_path.split('/')[-1]
        
        # Parse sensing start time (4th segment after splitting by '_')
        try:
            # Split: S3B_SL_2_LST____20231029T203741_20231029T204041_...
            time_segment = filename.split("____")[1]
            sensing_start = time_segment.split("_")[0]  # 20231029T203741
            file_date = datetime.strptime(sensing_start[:8], "%Y%m%d")
            
            if start_date <= file_date <= end_date:
                filtered_urls.append(http_url)
                
        except (IndexError, ValueError) as e:
            print(f"Skipping malformed filename: {filename}")
            continue

print(f"Found {len(filtered_urls)} datasets between {start_date.date()} and {end_date.date()}")


Found 1050 datasets between 2023-06-01 and 2023-10-30


In [3]:
spatial_extent = {
    'west': 3.3,
    'south': 50.7,
    'east': 7.2,
    'north': 53.6
}

bbox = [3.3, 50.7, 7.2, 53.6] 

In [4]:
def get_dataset_shape(url):
    """Get the shape of a Sentinel-3 SLSTR dataset"""
    try:
        ds = xr.open_dataset(
            url,
            engine="zarr",
            group="measurements",
            chunks={}
        )
        shape = ds['lst'].shape
        ds.close()
        return shape
    except Exception as e:
        print(f"Error getting shape for {url}: {e}")
        return None

def group_urls_by_shape(url_list):
    """Group URLs by their dataset shapes"""
    shape_groups = defaultdict(list)
    
    for i, url in enumerate(url_list):
        print(f"Checking shape for URL {i+1}/{len(url_list)}")
        shape = get_dataset_shape(url)
        if shape:
            shape_groups[shape].append(url)
    
    return dict(shape_groups)

In [5]:
def extract_date_from_s3_filename(url):
    """Extract sensing start date from Sentinel-3 SLSTR filename"""
    filename = url.split('/')[-1]
    
    # Split by four underscores: S3X_SL_2_LST____YYYYMMDDTHHMMSS_...
    parts = filename.split('____')
    if len(parts) > 1:
        time_segment = parts[1]
        sensing_start = time_segment.split('_')[0]  # Gets "20230614T202459"
        
        # Extract date part (first 8 characters)
        date_str = sensing_start[:8]  # Gets "20230614"
        date_obj = datetime.strptime(date_str, "%Y%m%d").date()
        return date_obj
    else:
        return None

def group_urls_by_date(url_list):
    """Group URLs by their acquisition date"""
    date_groups = defaultdict(list)
    
    for url in url_list:
        date = extract_date_from_s3_filename(url)
        if date:
            date_groups[date].append(url)
    
    return dict(date_groups)


# Combined Date and Shape Grouping
Since we’ve already discovered that shape-based grouping resolves dimension conflicts, we can combine both approaches for optimal results:

In [6]:
def group_by_date_and_shape(url_list):
    """Group URLs by both date and shape for maximum efficiency"""
    date_groups = group_urls_by_date(url_list)
    
    final_groups = {}
    
    for date, urls in date_groups.items():
        # Further group by shape within each date
        shape_groups = group_urls_by_shape(urls)
        
        for shape, shape_urls in shape_groups.items():
            group_key = f"{date}_{shape}"
            final_groups[group_key] = shape_urls
    
    return final_groups


In [11]:
import xarray as xr
from datetime import datetime, timedelta

def process_daily_groups(date_groups, bbox):
    """Process each date group separately"""
    daily_results = {}
    
    for date, urls in date_groups.items():
        print(f"Processing {len(urls)} files for {date}")
        
        # Filter by shape within date group if needed
        shape_groups = group_urls_by_shape(urls)
        
        # Process the largest shape group (most files)
        largest_group = max(shape_groups.values(), key=len)
        
        def extract_time_and_filter(ds):
            # Add time coordinate for this specific date
            ds = ds.assign_coords(time=datetime.combine(date, datetime.min.time()))
            
            # Apply spatial filtering for Netherlands
            mask = (
                (ds.latitude >= bbox[1]) & 
                (ds.latitude <= bbox[3]) &
                (ds.longitude >= bbox[0]) & 
                (ds.longitude <= bbox[2])
            )
            return ds.where(mask)
        
        # Process daily data
        daily_data = xr.open_mfdataset(
            largest_group,
            engine="zarr",
            group="measurements",
            preprocess=extract_time_and_filter,
            combine="nested",
            concat_dim="time",
            chunks="auto",
            parallel=True,
            consolidated=True
            
        )['lst']
        
        # Take daily maximum (or mean) to reduce to single value per day
        daily_results[date] = daily_data.max(dim="time")
    
    return daily_results


In [12]:
def create_time_series_from_daily_results(daily_results):
    """Combine daily results into time series"""
    # Sort by date and combine
    sorted_dates = sorted(daily_results.keys())
    
    datasets = []
    for date in sorted_dates:
        data = daily_results[date]
        # Add time dimension back
        data = data.expand_dims(time=[datetime.combine(date, datetime.min.time())])
        datasets.append(data)
    
    # Combine into time series
    time_series = xr.concat(datasets, dim="time")
    return time_series.sortby("time")


In [13]:
# Step 1: Group by date first
date_groups = group_urls_by_date(filtered_urls)

# Step 2: Process each date group
daily_lst_data = process_daily_groups(date_groups, bbox=[3.3, 50.7, 7.2, 53.6])

# Step 3: Create time series
lst_time_series = create_time_series_from_daily_results(daily_lst_data)




Processing 7 files for 2023-06-01
Checking shape for URL 1/7
Checking shape for URL 2/7
Checking shape for URL 3/7
Checking shape for URL 4/7
Checking shape for URL 5/7
Checking shape for URL 6/7
Checking shape for URL 7/7
Processing 6 files for 2023-06-02
Checking shape for URL 1/6
Checking shape for URL 2/6
Checking shape for URL 3/6
Checking shape for URL 4/6
Checking shape for URL 5/6
Checking shape for URL 6/6
Processing 8 files for 2023-06-03
Checking shape for URL 1/8
Checking shape for URL 2/8
Checking shape for URL 3/8
Checking shape for URL 4/8
Checking shape for URL 5/8
Checking shape for URL 6/8
Checking shape for URL 7/8
Checking shape for URL 8/8
Processing 8 files for 2023-06-04
Checking shape for URL 1/8
Checking shape for URL 2/8
Checking shape for URL 3/8
Checking shape for URL 4/8
Checking shape for URL 5/8
Checking shape for URL 6/8
Checking shape for URL 7/8
Checking shape for URL 8/8
Processing 7 files for 2023-06-05
Checking shape for URL 1/7
Checking shape for U

# Load and process the stack

In [ ]:
lst_stack = xr.open_mfdataset(
    filtered_urls,
    engine="zarr",
    group="measurements",
    preprocess=preprocess,
    combine="nested",
    concat_dim="time",
    chunks={},
    parallel=True
)


In [14]:
lst_time_series

<xarray.DataArray 'lst' (time: 151, rows: 1200, columns: 1500)> Size: 1GB
dask.array<getitem, shape=(151, 1200, 1500), dtype=float32, chunksize=(1, 1200, 1500), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1kB 2023-06-01 2023-06-02 ... 2023-10-29
Dimensions without coordinates: rows, columns

In [15]:
ds = xr.open_datatree(filtered_urls[0], engine="zarr",consolidated=True, chunks={})

In [16]:
ds

<xarray.DataTree>
Group: /
│   Attributes:
│       other_metadata:  {'L0_offset_between_scan_index_and_ISP_scan_count_in': 4...
│       stac_discovery:  {'assets': {}, 'bbox': [30.0789, 49.3816, 1.64856, 62.92...
├── Group: /conditions
│   ├── Group: /conditions/auxiliary
│   │   │   Dimensions:     (rows: 1200, columns: 1500, nval: 256)
│   │   │   Dimensions without coordinates: rows, columns, nval
│   │   │   Data variables:
│   │   │       biome       (rows, columns) uint8 2MB dask.array<chunksize=(1200, 1500), meta=np.ndarray>
│   │   │       elevation   (rows, columns) float64 14MB dask.array<chunksize=(1200, 1500), meta=np.ndarray>
│   │   │       fraction    (rows, columns) float32 7MB dask.array<chunksize=(1200, 1500), meta=np.ndarray>
│   │   │       ndvi        (rows, columns) float32 7MB dask.array<chunksize=(1200, 1500), meta=np.ndarray>
│   │   │       tcwv        (rows, columns) float32 7MB dask.array<chunksize=(1200, 1500), meta=np.ndarray>
│   │   │       validation  (nval) int8 256B dask.array<chunksize=(256,), meta=np.ndarray>
│   │   └── Group: /conditions/auxiliary/orphan
│   │           Dimensions:    (rows: 1200, orphan_pixels: 187)
│   │           Dimensions without coordinates: rows, orphan_pixels
│   │           Data variables:
│   │               biome      (rows, orphan_pixels) uint8 224kB dask.array<chunksize=(1200, 187), meta=np.ndarray>
│   │               elevation  (rows, orphan_pixels) float32 898kB dask.array<chunksize=(1200, 187), meta=np.ndarray>
│   │               fraction   (rows, orphan_pixels) float32 898kB dask.array<chunksize=(1200, 187), meta=np.ndarray>
│   │               ndvi       (rows, orphan_pixels) float32 898kB dask.array<chunksize=(1200, 187), meta=np.ndarray>
│   │               tcwv       (rows, orphan_pixels) float32 898kB dask.array<chunksize=(1200, 187), meta=np.ndarray>
│   ├── Group: /conditions/geometry
│   │       Dimensions:           (rows: 1200, columns: 130)
│   │       Coordinates:
│   │           x                 (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           y                 (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │       Dimensions without coordinates: rows, columns
│   │       Data variables:
│   │           sat_azimuth_tn    (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           sat_path_tn       (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           sat_zenith_tn     (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           solar_azimuth_tn  (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           solar_path_tn     (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           solar_zenith_tn   (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   ├── Group: /conditions/meteorology
│   │       Dimensions:                       (rows: 1200, columns: 130, t_series: 5,
│   │                                          p_atmos: 25)
│   │       Coordinates:
│   │           latitude                      (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           longitude                     (rows, columns) float64 1MB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │         * p_atmos                       (p_atmos) float32 100B 1.0 2.0 ... 950.0 1e+03
│   │         * t_series                      (t_series) datetime64[ns] 40B 2023-05-31T18...
│   │       Dimensions without coordinates: rows, columns
│   │       Data variables: (12/22)
│   │           cloud_fraction_tp             (rows, columns) float32 624kB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           dew_point_tp                  (rows, columns) float32 624kB dask.array<chunksize=(1200, 130), meta=np.ndarray>
│   │           east_west_stress